In [1]:
import xarray as xr
import numpy as np
import regionmask
from geocat.comp import eofunc_eofs, eofunc_pcs
import xesmf as xe

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('default')
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import cmasher as cmr
mpl.rcParams['hatch.linewidth'] = 0.5
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import pdo_functions
import importlib
importlib.reload(pdo_functions)
import glob
import re

In [5]:
# File list
start_year = 1850
end_year = 1859
file_list = sorted(glob.glob(f'/hurrell-scratch2/ivyglade/pdo/CESM2LE/SST/b.e21.BHISTsmbb.f09_g17.*.pop.h.SST.{start_year}01-{end_year}12.nc'))

# Extract ensemble member ID using regex
ids = [re.search(r'LE2-\d{4}\.\d{3}', f).group(0) for f in file_list]

# To extract lat/lon coords from
atm = xr.open_dataset('/scratch/ivyglade/pdo/CESM2LE/b.e21.BSSP370cmip6.f09_g17.LE2-1181.010.cam.h1.wrf_cape_2d.20150101-20241231.nc')

In [6]:
ids

['LE2-1011.001',
 'LE2-1031.002',
 'LE2-1051.003',
 'LE2-1071.004',
 'LE2-1091.005',
 'LE2-1111.006',
 'LE2-1131.007',
 'LE2-1151.008',
 'LE2-1171.009',
 'LE2-1191.010',
 'LE2-1231.011',
 'LE2-1231.012',
 'LE2-1231.013',
 'LE2-1231.014',
 'LE2-1231.015',
 'LE2-1231.016',
 'LE2-1231.017',
 'LE2-1231.018',
 'LE2-1231.019',
 'LE2-1231.020',
 'LE2-1251.011',
 'LE2-1251.012',
 'LE2-1251.013',
 'LE2-1251.014',
 'LE2-1251.015',
 'LE2-1251.016',
 'LE2-1251.017',
 'LE2-1251.018',
 'LE2-1251.019',
 'LE2-1251.020',
 'LE2-1281.011',
 'LE2-1281.012',
 'LE2-1281.013',
 'LE2-1281.014',
 'LE2-1281.015',
 'LE2-1281.016',
 'LE2-1281.017',
 'LE2-1281.018',
 'LE2-1281.019',
 'LE2-1281.020',
 'LE2-1301.011',
 'LE2-1301.012',
 'LE2-1301.013',
 'LE2-1301.014',
 'LE2-1301.015',
 'LE2-1301.016',
 'LE2-1301.017',
 'LE2-1301.018',
 'LE2-1301.019',
 'LE2-1301.020']

In [ ]:
def regrid_SST(file, atm_file):
    data = xr.open_dataset(file_list[0])

    # Extract SST and lat/lon coords
    sst = data['SST']
    lon = data['TLONG']
    lat = data['TLAT']

    # Creating an input grid
    grid_in = xr.Dataset({
        'lon': lon,
        'lat': lat, 
    })

    # Create output grid based on atm grid
    lat_out = atm_file['lat'].data
    lon_out = atm_file['lon'].data

    grid_out = xr.Dataset({
        'lat': (['lat'], lat_out),
        'lon': (['lon'], lon_out)
    })

    # regrid
    regridder = xe.Regridder(grid_in, grid_out, method='bilinear', periodic=True)

    regridded = regridder(sst)
    
    return regridded

In [ ]:
for i in range(50):
    regridded_sst = regrid_SST(file_list[i], atm)

    regridded_sst.to_netcdf(f'/hurrell-scratch2/ivyglade/pdo/CESM2LE/regridded_SST/b.e21.BSSP370cmip6.f09_g17.{ids[i]}.pop.h.regridded_SST.{start_year}01-{end_year}12.nc')

    print(f'ensemble member {i+1} is complete.')